## tf-idf cosine score calculations

## Step 0 : Using summary files, get info such as
### N ---> total number of patents
### ||V|| ---> vocabulary size
### list of all patents_ids
### list of all words
### inverted index for each word indicating the counts of occurence in various patent_ids

In [92]:
%%writefile tf.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-


from __future__ import division
#import collections
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
from nltk.corpus import stopwords

class MRtf(MRJob):
    OUTPUT_PROTOCOL = RawProtocol

    def mapper1(self, _, line):

        line = line.strip().lower()
        parts = line.split('\t')
        patent_id=parts[1]
        ## to get total number of docs
        yield "*"+patent_id,1
        
        ## to start building the inverted index 
        words=parts[2].split()
        for word in words:
            word=re.sub('[^a-z]+', '', word)
            if len(word)>0 and word not in stopwords.words('english'):
                yield word+"|"+patent_id,str(1)
                yield "!!total_vocabulary"+"|"+word,str(1)
    
    
    def reducer1(self, key, values):
        yield key, str(sum((int(v)) for v in values))

    
    def mapper2(self, key, value):
        #line=line.strip()
        #key,value=line.split('\t')
        if key[0]=="*":
            yield "**list_of_patents",key[1:]
            yield "**total_patents",str(1)
        elif key[0]=="!":
            yield "!!total_vocabulary",str(1)
            yield "!!list_of_words",key.split("|")[1]
        else:
            word,patent=key.split('|')
            yield word, patent+":"+str(value)
        
        
    def reducer2(self, key, values):
        if key=="**total_patents" or key=="!!total_vocabulary":
            yield key, str(sum((int(v)) for v in values))
        elif key=="**list_of_patents" or key=="!!list_of_words":
            yield key, ",".join(v for v in values)
        else:
            prev_doc=None
            prev_key=None
            temp_value=0
            temp_str=None
            for v in values:
                doc,count=v.split(":")
                if prev_key is None:
                    prev_key=key


                
                if prev_key==key:
                    
                
                    if prev_doc is not None:
                        if doc==prev_doc:
                    
                            temp_value+=int(count)
                        else:
                        
                            #yield prev_key,prev_doc+":"+str(temp_value)
                            if temp_str is not None:
                                temp_str=temp_str+","+prev_doc+":"+str(temp_value)
                            else:
                                temp_str=prev_doc+":"+str(temp_value)
                            
                            temp_value=int(count)
                            prev_key=key
                            prev_doc=doc
                    else:
                        temp_value=int(count)
                        prev_key=key
                        prev_doc=doc
                        temp_str=prev_doc+":"+str(temp_value)
                else:
                    
                    yield prev_key,temp_str
                    prev_key=key
                    temp_value=int(count)
                    prev_doc=doc
                    temp_str=prev_doc+":"+str(temp_value)
            yield prev_key,temp_str

                    
    def steps(self):

        JOBCONF_STEP1 = { 
            #'mapreduce.job.maps':'3',

            #'mapreduce.job.reduces': '1'
        }
        JOBCONF_STEP2 = {
             
            #'mapreduce.job.reduces': '1'
        }


        return [MRStep(jobconf=JOBCONF_STEP1,
                    mapper=self.mapper1 ,
                    reducer=self.reducer1)
                ,
                MRStep(jobconf=JOBCONF_STEP2,
                    mapper=self.mapper2 ,
                    reducer=self.reducer2)
                                ,
                ]
 

if __name__ == '__main__':
    MRtf.run() 

Overwriting tf.py


In [46]:
## we need to remove the headers
#!head -100 brf_sum_text.csv  |tail -99 >brf_sum_text_mini.csv

In [89]:
## we need to remove the headers
!tail -1238 brf_sum_text.csv >brf_sum_text_headerless.csv


In [94]:
!python tf.py -r local brf_sum_text_headerless.csv >brf_sum_text_output1.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.154811.503598
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.154811.503598/output...
Removing temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.154811.503598...


In [93]:
!python tf.py -r local brf_sum_text_mini.csv 

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.154751.707761
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.154751.707761/output...
!!list_of_words	ability,abundantly,accurate,activitiesthe,acuminate,acutely,advantage,advantageous,aforementioned,aforesaid,african,agriculturist,almond,also,alternate,among,andagainst,andin,appearance,appeared,approximate,april,asexual,asexually,attendant,attractive,atwater,august,average,b,background,basic,bearer,bearing,behalf,believed,blanched,blend,bloom,bloomer,blooming,blooms,borne,botanically,bouquet,branched,breeding,bud,budding,bush,calif,cameo,canes,careful,carried,carries,carryover,center,certain,chance,characteristically,characteristics,characterized,chula,class,classification,clean,climate,close,closely,coarsely,color,combination,come,commercial

In [32]:
!python tf.py -r local brf_sum_text_mini.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.032345.980818
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/tf.arvinsahni.20170327.032345.980818/output...
"!!total_vocabulary"	"ability:1"
"!!total_vocabulary"	"abundantly:2"
"!!total_vocabulary"	"accurate:1"
"!!total_vocabulary"	"activitiesthe:1"
"!!total_vocabulary"	"acuminate:1"
"!!total_vocabulary"	"acutely:1"
"!!total_vocabulary"	"advantage:1"
"!!total_vocabulary"	"advantageous:2"
"!!total_vocabulary"	"aforementioned:1"
"!!total_vocabulary"	"aforesaid:1"
"!!total_vocabulary"	"african:2"
"!!total_vocabulary"	"agriculturist:1"
"!!total_vocabulary"	"almond:7"
"!!total_vocabulary"	"also:1"
"!!total_vocabulary"	"alternate:1"
"!!total_vocabulary"	"among:1"
"!!total_vocabulary"	"andagainst:1"
"!!total_vocabulary"	"andin:1"
"!!total_vocabulary"	"appearance:2"
"!!tot

## Step 1 : Accept a patent_id as a query .Pass it as a parameter

## (a) Calculate Weight due to tf 
#### =1+log(base 10) of tf (t,d) ----> if tf(t,d) >0
#### =0 otherwise
---> will replace count by this value

## (b) Calculate Weight due to idf
#### =log(base 10) of N/ df(for term t ) 
#### here we will use N-1 as the query id is also included in the collected
----> add an additional value per word

## Score of tf-idf 
#### = sum( a*b) for all terms in both query and document






In [321]:
!head -2968 brf_sum_text_output1.csv > brf_sum_text_output1_mini.csv

In [322]:
## for some reason line 2969 on brf_sum_text_output1.csv is causing issues so lets try without that line
# !cp brf_sum_text_output1.csv brf_sum_text_output1_copy.csv
# removed blades on 2969 of brf_sum_text_output1.csv 

In [326]:
%%writefile scoring.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-


from __future__ import division
#import collections
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
from nltk.corpus import stopwords

class MRscoring(MRJob):
    OUTPUT_PROTOCOL = RawProtocol

    INTERNAL_PROTOCOL = RawProtocol

    def mapper1(self, _, line):

        line = line.strip()
        key,value=line.split('\t')
        if key=="!!total_vocabulary":
            self.vocab=int(value)
            yield key,value
       
        elif key=="!!list_of_words" or key =="**list_of_patents":
            #yield key,value
             yield key,str(0)   
        elif key=="**total_patents":
            self.collection_size=int(value)
            yield key,value

        else:
            value_parts=value.split(",")
            no_of_patents=int(len(value_parts))
            idf=np.log10(self.collection_size/no_of_patents)
            #yield key,str(idf)
            temp_str=None
            
            for v in value_parts:
                doc,count=v.split(":")

                count=1+np.log10(int(count))
                if doc==str(self.options.qp):        
                    
                    #yield "**query_word,tf,idf",key+":"+str(count*idf)+":"+str(idf)
                    yield "**query_word,tf,idf",key+":"+str(count*idf)
                    
                if temp_str is None:
                    temp_str=doc+":"+str(count*idf)
                else:
                    temp_str=temp_str+","+doc+":"+str(count*idf)
            #yield key,temp_str+"|"+str(idf)
            yield key,temp_str
        

    def reducer1(self,key,values):
        if key=="**query_word,tf,idf":
            yield key, ','.join(v for v in values)
        else:
            for v in values:
                yield key, str(v)
    
    
    ## Till now what we have is the query_word list with tf*idf and all other words with the list of docs 
    # and the corresponding tf*idf
    
    ## we need to get product of q*d values and the length of q and d
    ## to get the potential candidate list of d , we look at the words in query_words
    # amalgamate those and then maybe look at those with more than 10 common words. For testing this is set to 1 
    # , find a way to link it to length of Q
    
    def mapper2_init(self):
        self.query_words={}
    def mapper2(self,key,value):
        #query_words={}
        
        if key=="**query_word,tf,idf":
            temp=value.split(",")
            for t in temp:
                self.query_words[t.split(':')[0]]=t.split(':')[1]
                
            yield key,value
        elif key[0]=="*" or key[0]=="!":
            yield key,value
        else:
            
            if str(key) in self.query_words.keys():
                for v in value.split(","):
                    yield "**candidate_list"+"|"+v.split(":")[0],str(1)
                
                
            yield key,value
            
    def reducer2(self,key,values):
        if key.split("|")[0]=="**candidate_list":
            sum_temp=sum(int(v) for v in values)
            if sum_temp>3:
                yield key, str(sum_temp)
        
        else:
            for v in values:
                yield key, str(v)

    ## what we have now is a list of candidates with atleast a minimum common words
    ## next step is to change back the inversion to have data per document
    ## for normalization
    def configure_options(self):
        super(MRscoring, self).configure_options()
        
        self.add_passthrough_option('--query_patent', dest='qp', default="4490986",type='string',help="""patent_id""")
 
                   
    def steps(self):

        JOBCONF_STEP1 = { 
            #'mapreduce.job.maps':'3',

            #'mapreduce.job.reduces': '1'
            "mapreduce.map.memory.mb": "4096",
            "mapreduce.map.java.opts": "-Xmx6144m",
            "mapreduce.reduce.memory.mb": "4096",
            "mapreduce.reduce.java.opts": "-Xmx6144m"

        }
        JOBCONF_STEP2 = {
             
            #'mapreduce.job.reduces': '1'
        }


        return [MRStep(jobconf=JOBCONF_STEP1,
                    mapper=self.mapper1 ,
                    reducer=self.reducer1)
                ,
                MRStep(jobconf=JOBCONF_STEP2,
                    mapper_init=self.mapper2_init,   
                    mapper=self.mapper2 ,
                    reducer=self.reducer2
                      )
                               ,
                ]
 

if __name__ == '__main__':
    MRscoring.run() 

Overwriting scoring.py


In [328]:
!python scoring.py -r local brf_sum_text_output1_mini.csv >brf_sum_text_output2_mini.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/scoring.arvinsahni.20170328.115739.641541
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/scoring.arvinsahni.20170328.115739.641541/output...
Removing temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/scoring.arvinsahni.20170328.115739.641541...


In [246]:
#!head -5000 brf_sum_text_output1.csv > brf_sum_text_output1_mini.csv

In [ ]:
#!python scoring.py -r local brf_sum_text_output1_mini.csv >brf_sum_text_output2.csv

In [ ]:
## reverse the inversion of the candidate lists 

In [383]:
%%writefile reverse_inversion.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-


from __future__ import division
#import collections
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
from nltk.corpus import stopwords

class MRrevinversion(MRJob):
    OUTPUT_PROTOCOL = RawProtocol

    INTERNAL_PROTOCOL = RawProtocol

    def mapper1_init(self):
        self.query_words={}
        self.candidate_patents=[]

    def mapper1(self, _, line):

        line = line.strip()
        key,value=line.split('\t')
               
        if key=="!!list_of_words" or key =="**list_of_patents" or key =="**total_patents" or key=="!!total_vocabulary":
            #yield key,value
             yield key,str(0)   
        elif key=="**query_word,tf,idf":
            temp=value.split(",")
            for t in temp:
                self.query_words[t.split(':')[0]]=t.split(':')[1]
            yield key,value
        elif key[0:16]=="**candidate_list":
            self.candidate_patents.append(key.split("|")[1])
            
        else:
            value_parts=value.split(",")
            
           
            
            for v in value_parts:
                doc,tfidf=v.split(":")

                if doc in self.candidate_patents:
                    if key in self.query_words.keys():
                        temp=self.query_words[key]
                    else:
                        temp=0
                    yield doc,key+":"+str(tfidf)+"|"+str(temp)
                

    def reducer1(self,key,values):
        if key=="!!list_of_words" or key =="**list_of_patents" or key =="**total_patents" or key=="!!total_vocabulary":
            for v in values:
                yield key, str(v)
        elif key=="**query_word,tf,idf":
            for v in values:
                yield key,str(v)
        else:
            
            prev_key=None
            temp_value=0
            temp_str=None
            for v in values:

                if prev_key is None:
                    prev_key=key
                
                if prev_key==key:
                    if temp_str is not None:
                        temp_str=temp_str+","+str(v)
                    else:
                        temp_str=str(v)

                else:
                    
                    yield prev_key,temp_str
                    prev_key=key

                    temp_str=str(v)
            yield prev_key,temp_str 
    
    def mapper2(self,key,value):
        if key=="!!list_of_words" or key =="**list_of_patents" or key =="**total_patents" or key=="!!total_vocabulary" or key=="**query_word,tf,idf":
            yield key,str(value)
        else:
            l2_norm=0
            d_mult_q=0
            for v in value.split(","):
                v_d_q=v.split("|")[1]
                v_d=(v.split("|")[0]).split(":")[1]
                l2_norm=l2_norm+float(v_d)*float(v_d)
                d_mult_q=d_mult_q+float(v_d)*float(v_d_q)
            yield str(float(d_mult_q)/np.sqrt(l2_norm)),key
                   
    def steps(self):

        JOBCONF_STEP1 = { 
            #'mapreduce.job.maps':'3',

            #'mapreduce.job.reduces': '1'
        }

        JOBCONF_STEP2 = {
             
            #'mapreduce.job.reduces': '1'
        }


        return [MRStep(jobconf=JOBCONF_STEP1,
                    mapper_init=self.mapper1_init,   
                    mapper=self.mapper1,
                    reducer=self.reducer1)

                               ,
                MRStep(jobconf=JOBCONF_STEP2,
                    #mapper_init=self.mapper2_init,   
                    mapper=self.mapper2 #,
                    #reducer=self.reducer2
                      )
                               ,
                ]
 

if __name__ == '__main__':
    MRrevinversion.run() 

Overwriting reverse_inversion.py


In [384]:
!head -200 brf_sum_text_output2_mini.csv >brf_sum_text_output2_mini_mini.csv
!python reverse_inversion.py -r local brf_sum_text_output2_mini.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/reverse_inversion.arvinsahni.20170328.140518.292613
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/reverse_inversion.arvinsahni.20170328.140518.292613/output...
!!list_of_words	0
!!total_vocabulary	0
**list_of_patents	0
**query_word,tf,idf	ability:1.26021173198,abnormal:4.11498650546,abnormal:4.11498650546,absence:1.43950813091,accordance:0.570276411178,accumulated:1.79169064902,actual:1.11044941164,actuated:1.40252456466,addition:0.764447061512,additional:0.723504787274,airspace:4.02372232694,ajar:3.09272064468,alarm:2.49359232127,alarms:2.61559938996,alarms:2.61559938996,alert:2.49066065336,alert:2.49066065336,alerted:3.63207327299,alerted:3.63207327299,alerts:3.09272064468,allows:1.08412047292,also:0.326787690782,another:0.382593889835,apparatus:0.593033562066,apparent:0.672764896194,

In [395]:
%%writefile final_output.csv
0.385560792258	4490862
0.565840712083	4490883
1.10031631256	4490888
0.693692915331	4490918
0.693829738054	4490921
0.763762712024	4490935
0.55955694884	4490942
1.84576430445	4490946
0.847622262276	4490952
0.195891459781	4490968
0.460028573633	4490971
0.568522401323	4490974
11.8601700868	4490986
0.470723294018	4490994
0.346589405953	4491000
0.375937105203	4491006
0.199072319467	4491016
0.612928683478	4491022
0.649440071099	4491028
1.29292579965	4491043
0.450748609519	4491050
0.523381355316	4491061
0.307888776937	4491087
3.4670567536	4491103
1.5671054466	4491104
1.55446214105	4491126
1.74258375578	4491131
0.563766437444	4491140
0.683110275862	4491159
0.585378624381	4491183
1.56414926652	4491204
0.271583703798	4491210
0.99933299539	4491213
0.443711534232	4491225
0.554891462373	4491278
0.569101937733	4491296
0.326323057009	4491306
0.715599467477	4491307
0.222759340251	4491310
0.508401605981	4491314
0.474407776347	4491336
0.182837128728	4491367
0.323609872423	4491411
0.628682065793	4491424
0.264966176119	4491431
0.216017922316	4491444
0.27656529564	4491474
0.347325419753	4491479
0.253896805025	4491483
0.227328318417	4491488
0.746733146911	4491490
0.196968142373	4491508
0.270889961993	4491516
0.43960000685	4491519
0.203194955753	4491525
0.139237525625	4491535
0.329597746692	4491553
0.343522385092	4491570
0.516738172179	4491574
0.674494757919	4491589
0.678581801089	4491602
0.14667192392	4491609
0.335204760906	4491628
0.257682945564	4491632
0.503971675324	4491635
0.185134523094	4491639
0.248375878009	4491641
0.0925535596809	4491643
0.20077690651	4491646
0.323572078572	4491652
0.487082340757	4491653
0.260935900074	4491654
0.224721400854	4491671
0.798909461723	4491675
0.664051091534	4491690
0.338253485717	4491691
0.967127073813	4491712
0.810920479692	4491714
0.475337196246	4491718
1.86972184383	4491726
0.563926365671	4491727
0.511304065596	4491728
1.04013988319	4491733
0.443444015039	4491739
0.43285874602	4491757
0.500675584608	4491772
0.581009232518	4491773
0.94368298354	4491775
0.516948088169	4491777
0.769549535895	4491824
0.451712514422	4491836
0.189937463696	4491866
0.504547968462	4491895
0.666694968391	4491904
0.316224716461	4491910
0.290442897686	4491913
1.75702193909	4491933
0.265993605062	4491938
0.593780394728	4491945
0.448724862916	4491947
2.1162137712	4491966
0.819430852414	4491968

Overwriting final_output.csv


In [397]:
!sort -k1,1nr final_output.csv |cut -f 2 >sorted_predicts.txt 

In [398]:
!grep 4490986 uspatentcitation.csv | cut -f 3 >actual_citations.txt

In [364]:
#!python reverse_inversion.py -r local brf_sum_text_output2_mini.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/reverse_inversion.arvinsahni.20170328.131701.504433
Running step 1 of 1...
Streaming final output from /var/folders/d3/xgdrfhvs55x7kxm_pb89bgrr0000gn/T/reverse_inversion.arvinsahni.20170328.131701.504433/output...
!!list_of_words	0
!!total_vocabulary	0
**list_of_patents	0
**query_word,tf,idf	ability:1.26021173198,abnormal:4.11498650546,abnormal:4.11498650546,absence:1.43950813091,accordance:0.570276411178,accumulated:1.79169064902,actual:1.11044941164,actuated:1.40252456466,addition:0.764447061512,additional:0.723504787274,airspace:4.02372232694,ajar:3.09272064468,alarm:2.49359232127,alarms:2.61559938996,alarms:2.61559938996,alert:2.49066065336,alert:2.49066065336,alerted:3.63207327299,alerted:3.63207327299,alerts:3.09272064468,allows:1.08412047292,also:0.326787690782,another:0.382593889835,apparatus:0.593033562066,apparent:0.672764896194,assumes:2.01353939864,a